This is test file to store chat history and interact in long term

In [42]:
import os
from dotenv import load_dotenv

load_dotenv()

assert 'OPENAI_API_KEY' in os.environ, "You will need to set an OPENAI_API_KEY"
assert 'LANGCHAIN_API_KEY' in os.environ, "You will need to set an LANGCHAIN_API_KEY"

In [43]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.chains import LLMMathChain
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor

import random

In [44]:
llm = ChatOpenAI(temperature = 0, model='gpt-3.5-turbo-1106')
prompt = hub.pull("hwchase17/openai-functions-agent")

In [52]:
@tool
def rolld6() -> int:
    """ Roll a six-sided dice
    """
    rand = random.randint(1, 6)
    return rand

@tool
def rolld10() -> int:
    """ Roll a ten-sided dice
    """
    rand = random.randint(1, 10)
    return rand

@tool
def attack(attacker_power: int, defender_health: int) -> int:
    """ Attacker attacks the defender and check defener health
    args:
        attacker_power: power of the attack from the attacker
        defender_health: health of the defender
    """
    attacker_roll = rolld6({})
    attacker_trueap = attacker_power + attacker_roll
    defender_health -= attacker_trueap

    return defender_health

tools = [rolld6, rolld10, attack]

In [46]:
# Create an OpenAI Functions agent
agent = create_openai_functions_agent(llm, tools, prompt=prompt)
# Create an agent executor
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,tools=tools,verbose=True, max_iteration=50, max_execution_time=50, return_intermediate_steps=True)

In [47]:
# Test the agent
question = "Player rolls a d6, and NPC rolls a d6. Who wins?"
response = agent_executor.invoke({"input": question})
print(f"Question: {question}")
print(f"Answer: {response}")



> Entering new AgentExecutor chain...

Invoking: `rolld6` with `{}`


1
Invoking: `rolld6` with `{}`


1Both the player and the NPC rolled a 1 on a d6. This means it's a tie!

> Finished chain.
Question: Player rolls a d6, and NPC rolls a d6. Who wins?
Answer: {'input': 'Player rolls a d6, and NPC rolls a d6. Who wins?', 'output': "Both the player and the NPC rolled a 1 on a d6. This means it's a tie!", 'intermediate_steps': [(AgentActionMessageLog(tool='rolld6', tool_input={}, log='\nInvoking: `rolld6` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{}', 'name': 'rolld6'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_49836e94c4'}, id='run-70448435-53f7-4ade-a08c-9890a2f7e3fb')]), 1), (AgentActionMessageLog(tool='rolld6', tool_input={}, log='\nInvoking: `rolld6` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call':

In [56]:
from langchain_core.messages import AIMessage, HumanMessage
from IPython.display import clear_output


In [57]:

playing = True
chat_history = [
            HumanMessage(content="You are a DnD Dungeon Master. You will be responsible for writing a campagin to play a simplified DnD game on. Throughout the game, there are scenarios that requires a skill check, in which the player will roll a d10 by using the roll d10 tool. For easy scenarios, the player succeeds if they roll 3 or higher. For hard scenarios, the player succeeds if they roll 7 or higher. The player might also encounter combat scenarios, where the combat is played out by using the given attack tool")
        ]
while(playing):
    question = input()
    clear_output()

    if (len(question) < 1):
        playing = False

    response = agent_executor.invoke({
        "input" : question,
        "chat_history": chat_history,
    })

    print(f"Question: {question}")
    print(f"Answer: {response}")
    chat_history.append(HumanMessage(content=question))
    chat_history.append(AIMessage(content=response['output']))



> Entering new AgentExecutor chain...
It seems we have encountered a combat scenario. To resolve this, we will use the attack tool. I will roll a six-sided dice to determine the outcome of your attack against the wolves. Let's proceed with the combat.

> Finished chain.
Question: 
Answer: {'input': '', 'chat_history': [HumanMessage(content='You are a DnD Dungeon Master. You will be responsible for writing a campagin to play a simplified DnD game on. Throughout the game, there are scenarios that requires a skill check, in which the player will roll a d10 by using the roll d10 tool. For easy scenarios, the player succeeds if they roll 3 or higher. For hard scenarios, the player succeeds if they roll 7 or higher. The player might also encounter combat scenarios, where the combat is played out by using the given attack tool', additional_kwargs={}, response_metadata={}), HumanMessage(content='Who am I? Where am I?', additional_kwargs={}, response_metadata={}), AIMessage(content='You are a